In [2]:
import os
os.chdir("../..") # Change to parent directory
from struc2vec.src.Helpers import *
os.chdir("structuralNetworkMigration")

In [3]:
import statsmodels.api as sm
import numpy as np
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import re

# Classify the classifications
I will find the different structural features and development features of the three classifications, both in terms of mean and variance within the class.

## Load graph and classifications

In [4]:
G = nx.read_gexf("data/Graphs/bynode_0_directed.gexf")

In [5]:
with open("data/embeddings/node2class.json", "r") as f:
    class_data = json.load(f)

In [6]:
df_struc_feat, df_struc_feat_grouped = get_meta_data(G, class_data.keys(), class_data.values())

## Create table of structural features grouped by classification

In [7]:
df_struc_feat.describe().T

,count,mean,std,min,25%,50%,75%,max
Embedding,226.0,1.000000,0.858940,0.000000,0.000000,1.000000,2.000000,2.000000
Clustering,226.0,0.612844,0.161953,0.234189,0.528562,0.611871,0.703280,1.000000
Betweenness_centrality,226.0,0.003656,0.011166,0.000000,0.000022,0.000349,0.002011,0.088446
Closeness_centrality,226.0,0.447966,0.225726,0.000000,0.418802,0.501114,0.537956,0.918367
InDegree,226.0,36.712389,50.844701,0.000000,5.000000,17.000000,38.750000,205.000000
OutDegree,226.0,36.712389,23.397085,1.000000,20.000000,35.500000,51.000000,128.000000
Average_weight,226.0,12.846571,46.952251,0.021500,0.544010,1.556744,5.501317,452.658628


In [8]:
df_list = []
for class_, df_class in df_struc_feat.groupby("Embedding"):
    struc_data_ = df_class.drop("Embedding", axis=1).describe().T
    index_tup = [(feat_, class_) for feat_ in struc_data_.index]
    index_ = pd.MultiIndex.from_tuples(index_tup)
    struc_data_.index = index_
    df_list.append(struc_data_)

In [9]:
struc_feat = pd.concat(df_list)
index_sort = sorted(struc_feat.index)
latex_table = struc_feat.loc[index_sort].apply(lambda x: round(x,2)).to_latex()

## Create histograms

In [42]:
# fig, ax = plt.subplots(2,3, figsize=[17,10])
# sns.histplot(df_struc_feat[df_struc_feat["Embedding"] == 0]["Clustering"], ax=ax[0,0])
# ax[0,0].set_xlim(0,1)
# ax[0,0].set_ylim(ymin=0,ymax=None)
# ax[0,0].set_title("Class 0")
# ax[0,0].set_xlabel("Clust!")

In [49]:
plot_settings = {
    "Clustering":
        {"ymin":None,"ymax":25,
        "xmin":0,"xmax":1,
        "xlabel":"Clustering"},
    "Betweenness_centrality":
        {"ymin":None,"ymax":36,
        "xmin":0,"xmax":0.09,
        "xlabel": "Betweenness centrality"},
    "Closeness_centrality":
        {"ymin":None,"ymax":40,
        "xmin":0,"xmax":1,
        "xlabel":"Closeness centrality"},
    "InDegree":
        {"ymin":None,"ymax":35,
        "xmin":0,"xmax":200,
        "xlabel":"In-degree"},
    "OutDegree":
        {"ymin":None,"ymax":25,
        "xmin":0,"xmax":125,
        "xlabel":"Out-degree"},
    "Average_weight":
        {"ymin":None,"ymax":None,
        "xmin":None,"xmax":None,
        "xlabel":"Average weight"},
                  }

In [32]:
data_dict = {0:df_struc_feat[df_struc_feat["Embedding"] == 0],
             1: df_struc_feat[df_struc_feat["Embedding"] == 1],
             2: df_struc_feat[df_struc_feat["Embedding"] == 2]}

In [33]:
struc_feat_columns =list(df_struc_feat.columns)[2:]

In [52]:
# fig, ax = plt.subplots(6,3, figsize=[17,33])

# for i, column_ in enumerate(struc_feat_columns):
#     ymin = plot_settings[column_]["ymin"]
#     ymax = plot_settings[column_]["ymax"]
#     xmin = plot_settings[column_]["xmin"]
#     xmax = plot_settings[column_]["xmax"]
#     xlabel = plot_settings[column_]["xlabel"]
#     for j in range(3):
#         data_ = data_dict[j]
#         sns.histplot(data_[column_], ax=ax[i,j])
#         ax[i,j].set_title(f"Classification {j}")
#         ax[i,j].set_ylim(ymin=ymin,ymax=ymax)
#         ax[i,j].set_xlim(xmin=xmin,xmax=xmax)
#         ax[i,j].set_xlabel(xlabel)
# plt.savefig("visuals/struc_feat_histplot.jpg")
# plt.show()

## Describe development features

In [98]:
df_dev = pd.read_csv("data/metadata_country/metadata_full.csv").set_index("Node", drop=True)

In [119]:
df_dev["GDP (Billion $)"] = df_dev["GDP"] / 1000000000

In [120]:
df_dev["Class"] = df_dev["Class"].apply(int)

In [128]:
df_dev_econ = df_dev[["Class", "Human Development Index (value)", "Mean Years of Schooling (years)", "Expected Years of Schooling (years)", "GDP (Billion $)" ]]

In [143]:
df_dev_fert = df_dev.iloc[:,[0,-6,-5,-4,-3]]

In [144]:
df_dev_fert

,Class,"Fertility rate, total (births per woman)","Mortality rate, adult, female (per 1,000 female adults)","Mortality rate, adult, male (per 1,000 male adults)","Mortality rate, infant (per 1,000 live births)"
Node,,,,,
AFG,1,5.405,209.573,278.424,54.6
AUS,1,1.814,45.231,78.367,3.3
AUT,1,1.490,44.687,84.688,3.1
BHR,2,2.131,51.933,64.528,6.5
BEL,1,1.700,53.983,89.437,3.4
...,...,...,...,...,...
TON,0,3.562,156.780,237.489,10.7
TKM,0,2.930,120.273,236.782,36.7
UZB,1,2.491,121.508,212.748,17.7


In [145]:
cur_df_dev = df_dev_fert
df_list = []
for class_, df_class in cur_df_dev.groupby("Class"):
    dev_data_ = df_class.drop("Class", axis=1).describe().T
    index_tup = [(feat_, class_) for feat_ in dev_data_.index]
    index_ = pd.MultiIndex.from_tuples(index_tup)
    dev_data_.index = index_
    df_list.append(dev_data_)

In [146]:
dev_feat = pd.concat(df_list)
index_sort = sorted(dev_feat.index)
latex_table = dev_feat.loc[index_sort]

In [147]:
latex_table

count        mean  \
Fertility rate, total (births per woman)           0   63.0    2.966294   
                                                   1   60.0    2.099642   
                                                   2   83.0    3.010217   
Mortality rate, adult, female (per 1,000 female... 0   60.0  154.744700   
                                                   1   59.0   83.693847   
                                                   2   83.0  138.335108   
Mortality rate, adult, male (per 1,000 male adu... 0   60.0  229.009983   
                                                   1   59.0  147.790034   
                                                   2   83.0  212.108771   
Mortality rate, infant (per 1,000 live births)     0   51.0   31.339216   
                                                   1   60.0   12.341667   
                                                   2   82.0   25.320732   

                                                             std     min  \
Fertility rate, total (births per woman)           0    1.307041   1.186   
                                                   1    0.902832   1.240   
                                                   2    1.527698   1.196   
Mortality rate, adult, female (per 1,000 female... 0   99.954454  27.943   
                                                   1   59.782469  29.260   
                                                   2   84.953271  26.316   
Mortality rate, adult, male (per 1,000 male adu... 0  111.423115  54.909   
                                                   1   84.805747  53.663   
                                                   2   95.714617  50.600   
Mortality rate, infant (per 1,000 live births)     0   22.457614   1.800   
                                                   1   14.911903   2.000   
                                                   2   20.627608   2.100   

                                                            25%       50%  \
Fertility rate, total (births per woman)           0    1.90250    2.4940   
                                                   1    1.53750    1.7770   
                                                   2    1.86350    2.5340   
Mortality rate, adult, female (per 1,000 female... 0   73.06300  129.0455   
                                                   1   46.02650   59.7260   
                                                   2   73.77150  112.3490   
Mortality rate, adult, male (per 1,000 male adu... 0  162.18025  201.6935   
                                                   1   77.87300  121.6520   
                                                   2  142.10900  199.1500   
Mortality rate, infant (per 1,000 live births)     0   14.35000   25.3000   
                                                   1    3.20000    5.6000   
                                                   2    8.60000   17.5000   

                                                            75%      max  
Fertility rate, total (births per woman)           0    4.13200    6.035  
                                                   1    2.26675    5.405  
                                                   2    3.96950    7.211  
Mortality rate, adult, female (per 1,000 female... 0  213.87050  482.662  
                                                   1   93.06650  306.191  
                                                   2  193.70250  364.599  
Mortality rate, adult, male (per 1,000 male adu... 0  286.21275  593.377  
                                                   1  201.61400  394.214  
                                                   2  278.38950  454.913  
Mortality rate, infant (per 1,000 live births)     0   46.65000   95.300  
                                                   1   14.60000   71.700  
                                                   2   38.80000   82.700

In [148]:
s = latex_table.style

In [149]:
s.format({
    "count":'{:.0f}',
    }).format("{:.2f}",
        subset=latex_table.columns[1:]
    )

In [150]:
print(s.to_latex())

\begin{tabular}{llrrrrrrrr}
 &  & count & mean & std & min & 25% & 50% & 75% & max \\
\multirow[c]{3}{*}{Fertility rate, total (births per woman)} & 0 & 63 & 2.97 & 1.31 & 1.19 & 1.90 & 2.49 & 4.13 & 6.04 \\
 & 1 & 60 & 2.10 & 0.90 & 1.24 & 1.54 & 1.78 & 2.27 & 5.41 \\
 & 2 & 83 & 3.01 & 1.53 & 1.20 & 1.86 & 2.53 & 3.97 & 7.21 \\
\multirow[c]{3}{*}{Mortality rate, adult, female (per 1,000 female adults)} & 0 & 60 & 154.74 & 99.95 & 27.94 & 73.06 & 129.05 & 213.87 & 482.66 \\
 & 1 & 59 & 83.69 & 59.78 & 29.26 & 46.03 & 59.73 & 93.07 & 306.19 \\
 & 2 & 83 & 138.34 & 84.95 & 26.32 & 73.77 & 112.35 & 193.70 & 364.60 \\
\multirow[c]{3}{*}{Mortality rate, adult, male (per 1,000 male adults)} & 0 & 60 & 229.01 & 111.42 & 54.91 & 162.18 & 201.69 & 286.21 & 593.38 \\
 & 1 & 59 & 147.79 & 84.81 & 53.66 & 77.87 & 121.65 & 201.61 & 394.21 \\
 & 2 & 83 & 212.11 & 95.71 & 50.60 & 142.11 & 199.15 & 278.39 & 454.91 \\
\multirow[c]{3}{*}{Mortality rate, infant (per 1,000 live births)} & 0 & 51 & 31.34 

In [178]:
plot_settings = {
    "Fertility rate, total (births per woman)":
        {"ymin":None,"ymax":35,
        "xmin":1,"xmax":7},
    "Mortality rate, adult, female (per 1,000 female adults)":
        {"ymin":None,"ymax":40,
        "xmin":0,"xmax":500},
    "Mortality rate, adult, male (per 1,000 male adults)":
        {"ymin":None,"ymax":42,
        "xmin":0,"xmax":600},
    "Mortality rate, infant (per 1,000 live births)":
        {"ymin":None,"ymax":60,
        "xmin":0,"xmax":82},   
                  }

In [173]:
cur_df_dev = df_dev_fert
data_dict = {0:cur_df_dev[cur_df_dev["Class"] == 0],
             1: cur_df_dev[cur_df_dev["Class"] == 1],
             2: cur_df_dev[cur_df_dev["Class"] == 2]}

dev_columns =list(cur_df_dev.columns)[1:]


In [174]:
dev_columns

['Fertility rate, total (births per woman)',
 'Mortality rate, adult, female (per 1,000 female adults)',
 'Mortality rate, adult, male (per 1,000 male adults)',
 'Mortality rate, infant (per 1,000 live births)']

In [181]:
# fig, ax = plt.subplots(4,3, figsize=[17,25])

# for i, column_ in enumerate(dev_columns):
#     ymin = plot_settings[column_]["ymin"]
#     ymax = plot_settings[column_]["ymax"]
#     xmin = plot_settings[column_]["xmin"]
#     xmax = plot_settings[column_]["xmax"]
#     for j in range(3):
#         data_ = data_dict[j]
#         sns.histplot(data_[column_], ax=ax[i,j],stat="percent")
#         ax[i,j].set_title(f"Classification {j}, (n={sum(data_[column_].notna())})")
#         ax[i,j].set_ylim(ymin=ymin,ymax=ymax)
#         ax[i,j].set_xlim(xmin=xmin,xmax=xmax)
# plt.savefig("visuals/dev_fert_histplot.jpg")
# plt.show()